In [1]:
import pandas as pd
import json
pd.set_option('display.max_columns', None)


with open('../data/gsw_box_scores_full.json', encoding="utf8") as file:
    game_data = json.load(file)

**Process each player's stats for a game as an individual row**

In [2]:
def process_player_data(team_data, team_type, date_et, self_team, other_team):
    players = team_data['players']
    processed_data = []

    for player in players:
        player_data = {
            "ID": player.get('personId'),
            "NAME": player.get('firstName') + ' ' + player.get('familyName'),
            "TEAM": self_team,
            "OPP": other_team,
            "STATUS": team_type,
            "DATE": date_et,
        }

        player_data.update(player.get('statistics', {}))
        processed_data.append(player_data)

    return processed_data


def player_data_processor(game_data):
    all_players_data = []
    for game in game_data['games']:
        date_et = game['gameEt'][:10]
        away_team_code = game['awayTeam']['teamTricode']
        home_team_code = game['homeTeam']['teamTricode']
        away_team_data = process_player_data(game['awayTeam'], 'away', date_et, away_team_code, home_team_code)
        home_team_data = process_player_data(game['homeTeam'], 'home', date_et, home_team_code, away_team_code)
        all_players_data += away_team_data + home_team_data

    return all_players_data

**Create players stats spreadsheet**

In [6]:
all_players_data = player_data_processor(game_data=game_data)
df = pd.DataFrame(all_players_data)
print("Player Stats DataFrame:", df.shape)

df.to_excel('../data/gsw_box_score_player_stats.xlsx')

Player Stats DataFrame: (2175, 26)


**Process each team's total stats for a game as an individual row**

In [3]:
def process_team_data(team_data, team_type, date_et, self_team, other_team):
    team_stats = {
        "TEAM": self_team,
        "OPP": other_team,
        "STATUS": team_type,
        "DATE": date_et,
    }
    team_stats.update(team_data['statistics'])
    for i in range (4):
        team_stats[f'Q{i+1}'] = team_data['periods'][i]['score']

    return team_stats


def team_data_processor(game_data):
    all_team_data = []
    for game in game_data['games']:
        date_et = game['gameEt'][:10]
        away_team_code = game['awayTeam']['teamTricode']
        home_team_code = game['homeTeam']['teamTricode']
        away_team_data = process_team_data(game['awayTeam'], 'away', date_et, away_team_code, home_team_code)
        home_team_data = process_team_data(game['homeTeam'], 'home', date_et, home_team_code, away_team_code)
        away_team_data['W/L'] = 'W' if away_team_data['PTS'] > home_team_data['PTS'] else 'L'
        home_team_data['W/L'] = 'W' if home_team_data['PTS'] > away_team_data['PTS'] else 'L'
        all_team_data.append(away_team_data)
        all_team_data.append(home_team_data)

    return all_team_data

**Create team stats spreadsheet**

In [7]:
all_teams_data = team_data_processor(game_data=game_data)
df2 = pd.DataFrame(all_teams_data)
print("Team Stats DataFrame:", df2.shape)

df2.to_excel('../data/gsw_box_score_team_stats.xlsx')

Team Stats DataFrame: (164, 29)


---

## PLAYOFFS

**Create Playoffs players stats spreadsheet**

In [4]:
with open('../data/gsw_box_scores_playoffs_full.json', encoding="utf8") as file:
    playoffs_data = json.load(file)

In [5]:
all_playoffs_players_data = player_data_processor(game_data=playoffs_data)
df3 = pd.DataFrame(all_playoffs_players_data)
print("Playoffs Player Stats DataFrame:", df3.shape)

df3.to_excel('../data/gsw_box_score_player_playoffs_stats.xlsx')

Playoffs Player Stats DataFrame: (358, 26)


**Create Playoffs team stats spreadsheet**

In [6]:
all_playoffs_teams_data = team_data_processor(game_data=playoffs_data)
df4 = pd.DataFrame(all_playoffs_teams_data)
print("Playoffs Team Stats DataFrame:", df4.shape)

df4.to_excel('../data/gsw_box_score_team_playoffs_stats.xlsx')

Playoffs Team Stats DataFrame: (24, 29)


---